In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [1]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!, evaluate
using DiffResults
using Distributions
using Random
using TensorOperations
using Cubature
using LoopVectorization
using Optim
using SparseArrays

In [6]:
using ForwardDiff, Optim, SparseArrays
initial_x = zeros(100)
plap(U; n = length(U)) = (n-1)*sum((0.1 .+ diff(U).^2).^2 ) - sum(U) / (n-1)
plap1(x) = ForwardDiff.gradient(plap,x)
precond(n) = spdiagm(-1 => -ones(n-1), 0 => 2ones(n), 1 => -ones(n-1)) * (n+1)
f(x) = plap([0; x; 0])
g!(G, x) = copyto!(G, (plap1([0; x; 0]))[2:end-1])
result = Optim.optimize(f, g!, initial_x, method = ConjugateGradient(P = nothing))
result = Optim.optimize(f, g!, initial_x, method = ConjugateGradient(P = precond(100)))

MethodError: MethodError: no method matching ldiv!(::SparseMatrixCSC{Float64,Int64}, ::Array{Float64,1})
Closest candidates are:
  ldiv!(!Matched::IterativeSolvers.Identity, ::Any) at /home/mat/.julia/packages/IterativeSolvers/3g7hG/src/common.jl:31
  ldiv!(!Matched::Number, ::AbstractArray) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/LinearAlgebra/src/generic.jl:252
  ldiv!(::AbstractArray, !Matched::Number, !Matched::AbstractArray) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/LinearAlgebra/src/generic.jl:257
  ...

In [3]:
rosenbrock(x) =  (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
@time result = optimize(rosenbrock, zeros(2), LBFGS())

  1.548309 seconds (4.81 M allocations: 242.425 MiB, 4.08% gc time)


 * Status: success

 * Candidate solution
    Final objective value:     5.378388e-17

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.54e-11 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.54e-11 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.85e-19 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.30e-03 ≰ 0.0e+00
    |g(x)|                 = 9.88e-14 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    24
    f(x) calls:    67
    ∇f(x) calls:   67


In [8]:
function timing()
    result = optimize(rosenbrock, zeros(2), LBFGS())
end

timing (generic function with 1 method)

In [9]:
function f(x)
    return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end

function g!(G, x)
    G[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    G[2] = 200.0 * (x[2] - x[1]^2)
end

function h!(H, x)
    H[1, 1] = 2.0 - 400.0 * x[2] + 1200.0 * x[1]^2
    H[1, 2] = -400.0 * x[1]
    H[2, 1] = -400.0 * x[1]
    H[2, 2] = 200.0
end

initial_x = zeros(2)

2-element Array{Float64,1}:
 0.0
 0.0

In [17]:
@time Optim.minimizer(optimize(f, g!, h!, initial_x, BFGS()))

  0.000070 seconds (476 allocations: 20.172 KiB)


2-element Array{Float64,1}:
 0.9999999999373613
 0.9999999998686219

In [18]:
function fg!(F,G,x)
  # do common computations here
  # ...
  if G != nothing
    G[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    G[2] = 200.0 * (x[2] - x[1]^2)
    # code to compute gradient here
    # writing the result to the vector G
  end
  if F != nothing
    value = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
    # value = ... code to compute objective function
    return value
  end
end

fg! (generic function with 1 method)

In [34]:
x0 = zeros(2);

In [36]:
x0

2-element Array{Float64,1}:
 0.0
 0.0

In [51]:
res = Optim.optimize(Optim.only_fg!(fg!), x0, Optim.LBFGS())
@time Optim.minimizer(res)

  0.000003 seconds


2-element Array{Float64,1}:
 0.999999999999928
 0.9999999999998559

In [47]:
fieldnames(typeof(res))

(:method, :initial_x, :minimizer, :minimum, :iterations, :iteration_converged, :x_converged, :x_abstol, :x_reltol, :x_abschange, :x_relchange, :f_converged, :f_abstol, :f_reltol, :f_abschange, :f_relchange, :g_converged, :g_abstol, :g_residual, :f_increased, :trace, :f_calls, :g_calls, :h_calls, :ls_success, :time_limit, :time_run)

In [33]:
function fgh!(F,G,H,x)
    if F != nothing
        return f(x)
    end
    if G != nothing
        g!(G, x)
        return G
    end
    
    if H != nothing
        h!(H, x)
        return H
    end
end

Optim.optimize(Optim.only_fgh!(fgh!), [0., 0.], Optim.Newton())

 * Status: failure (objective increased between iterations)

 * Candidate solution
    Final objective value:     1.000000e+00

 * Found with
    Algorithm:     Newton's Method

 * Convergence measures
    |x - x'|               = 1.00e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.00e+00 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.00e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.00e-06 ≰ 0.0e+00
    |g(x)|                 = 8.80e-02 ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    71
    ∇f(x) calls:   71
    ∇²f(x) calls:  2


In [28]:
function timing()
    @btime optimize(rosenbrock, zeros(2), LBFGS())
    @btime Optim.optimize(Optim.only_fg!(fg!), [0., 0.], Optim.LBFGS())
    @btime Optim.optimize(Optim.only_fgh!(fgh!), [0., 0.], Optim.Newton())
end

timing (generic function with 1 method)

In [29]:
timing()

  42.254 μs (578 allocations: 27.69 KiB)
  37.231 μs (572 allocations: 27.59 KiB)


ArgumentError: ArgumentError: Value and slope at step length = 0 must be finite.